# WANN experiments on Kin familly dataset

In [1]:
import copy
import sys

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.constraints import MinMaxNorm

sys.path.append("../wann")
from utils import kin, BaggingModels, cross_val
from kin_experiments import run_kin_experiments
from methods import *

from warnings import filterwarnings
filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Load Dataset

In [2]:
source = 'kin-8fh'
target = 'kin-8fm'

N = 10   # Number of labeled target data
m = 200  # Number of labeled source data
n = 200  # Number of unlabeled target data

Xs, ys = kin(source)
Xt, yt = kin(target)
X = np.concatenate((Xs, Xt))
y = np.concatenate((ys, yt))

np.random.seed(0)
src_index = np.random.choice(len(Xs), m, replace=False)
tgt_index, tgt_test_index = train_test_split(range(len(Xs), len(Xs)+len(Xt)), train_size=n, test_size=2*n)
tgt_train_index = np.random.choice(tgt_index, N, replace=False)
train_index = np.concatenate((src_index, tgt_train_index))
shape = X.shape[1]

## Base Estimator

In [3]:
def get_base_model(shape, activation=None, C=1, name="BaseModel"):
    inputs = Input(shape=(shape,))
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    modeled = Dropout(0.5)(modeled)
    modeled = Dense(10, activation='relu',
                    kernel_constraint=MinMaxNorm(0, C),
                    bias_constraint=MinMaxNorm(0, C))(modeled)
    modeled = Dropout(0.2)(modeled)
    modeled = Dense(1, activation=activation,
                    kernel_constraint=MinMaxNorm(0, C),
                    bias_constraint=MinMaxNorm(0, C))(modeled)
    model = Model(inputs, modeled)
    model.compile(optimizer="adam", loss='mean_squared_error')
    return model

def get_encoder(shape, C=1, name="encoder"):
    inputs = Input(shape=(shape,))
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    modeled = Dropout(0.5)(modeled)
    modeled = Dense(10, activation='relu',
                    kernel_constraint=MinMaxNorm(0, C),
                    bias_constraint=MinMaxNorm(0, C))(modeled)
    modeled = Dropout(0.2)(modeled)
    model = Model(inputs, modeled)
    model.compile(optimizer="adam", loss='mean_squared_error')
    return model

def get_task(shape, C=1, activation=None, name="task"):
    inputs = Input(shape=(shape,))
    modeled = Dense(1, activation=activation,
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    model = Model(inputs, modeled)
    model.compile(optimizer="adam", loss='mean_squared_error')
    return model


base_estimator = BaggingModels(func=get_base_model,
                               n_models=1,
                               n_jobs=None,
                               shape=shape,
                               C=1,
                               random_state=0)
fit_params = dict(epochs=300,
                  batch_size=32,
                  verbose=0)

## Examples

### No reweight

In [4]:
no_reweight = copy.deepcopy(base_estimator)
no_reweight.fit(X[train_index], y[train_index], **fit_params)

y_pred = no_reweight.predict(X)
score= mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.4f'%score)

Target score: 0.0012


### TrAdaBoostR2

In [4]:
tradaboost = TwoStageTrAdaBoostR2(func=get_base_model,
                                  n_jobs=None,
                                  verbose=1,
                                  C=10,
                                  random_state=0,
                                  shape=X.shape[1])
tradaboost.fit(X, y, [src_index, tgt_train_index], **fit_params)
y_pred = tradaboost.predict(X)
score= mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.4f'%score)

cv error of estimator 0: 0.001 (0.0008711958)
cv error of estimator 1: 0.001 (0.0008832393)
cv error of estimator 2: 0.002 (0.0016080431)
cv error of estimator 3: 0.003 (0.0028950071)
cv error of estimator 4: 0.003 (0.0043617909)
cv error of estimator 5: 0.005 (0.0064908934)
cv error of estimator 6: 0.005 (0.0069557037)
cv error of estimator 7: 0.006 (0.0076525647)
cv error of estimator 8: 0.006 (0.0065013523)
cv error of estimator 9: 0.010 (0.0090407186)
Target score: 0.0010


### KMM

In [6]:
kmm = KMM(base_estimator, sigma=0.1)
kmm.fit(X, y, index=[src_index, tgt_index, tgt_train_index], **fit_params)

y_pred = kmm.predict(X)
score= mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.4f'%score)

Target score: 0.0021


### KLIEP

In [7]:
kliep = KLIEP(base_estimator)
kliep.fit(X, y, index=[train_index, tgt_index], **fit_params)

y_pred = kliep.predict(X)
score= mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.4f'%score)

Target score: 0.0012


### DANN

In [8]:
dann = BaggingModels(DANN, n_models=1, n_jobs=None, random_state=0,
                     get_encoder=get_encoder, get_task=get_task, C=1, lambda_=0.1)

dann.fit(X, y, index=[src_index, tgt_index, tgt_train_index], **fit_params)

y_pred = dann.predict(X)
score = mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.4f'%score)

Target score: 0.0007


### WANN

In [9]:
wann = BaggingModels(WANN, n_models=1, n_jobs=None, random_state=0,
                     get_base_model=get_base_model, C=1, C_w=1)
wann.fit(X, y, index=[src_index, tgt_train_index], **fit_params)

y_pred = wann.predict(X)
score = mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.4f'%score)

Target score: 0.0006


#### Cross Validation Cw Selection

In [10]:
cross_val("WANN", X, y, src_index, None, tgt_train_index,
          params=[0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1],
          fit_params=fit_params, cv=5,
          get_base_model=get_base_model, C=1);

Cross Validation: param = 0.010 | score = 0.0042
Cross Validation: param = 0.020 | score = 0.0037
Cross Validation: param = 0.050 | score = 0.0030
Cross Validation: param = 0.100 | score = 0.0024
Cross Validation: param = 0.200 | score = 0.0020
Cross Validation: param = 0.500 | score = 0.0012
Cross Validation: param = 1.000 | score = 0.0010
Best: param = 1.000 | score = 0.0010


## Run Experiments

In [11]:
df = run_kin_experiments(method="WANN",
                         get_base_model=get_base_model,
                         get_encoder=get_encoder,
                         get_task=get_task,
                         C=1,
                         C_w=1,
                         lambda_=0.1,
                         sigma=0.1,
                         epochs=300,
                         batch_size=32,
                         n_models=1,
                         n_jobs=None,
                         n_source=200,
                         n_target_unlabeled=200,
                         n_target_labeled=10,
                         n_target_test=400,
                         random_state=0,
                         save=False)

 
Experiment for method: WANN
 
############# kin-8fh #############
--------- kin-8fm ----------
Target_score: 0.0006
--------- kin-8nh ----------
Target_score: 0.0573
--------- kin-8nm ----------
Target_score: 0.0629
############# kin-8fm #############
--------- kin-8fh ----------
Target_score: 0.0019
--------- kin-8nh ----------
Target_score: 0.0536
--------- kin-8nm ----------
Target_score: 0.0601
############# kin-8nh #############
--------- kin-8fh ----------
Target_score: 0.0023
--------- kin-8fm ----------
Target_score: 0.0015
--------- kin-8nm ----------
Target_score: 0.0398
############# kin-8nm #############
--------- kin-8fh ----------
Target_score: 0.0024
--------- kin-8fm ----------
Target_score: 0.0010
--------- kin-8nh ----------
Target_score: 0.0366


### Launch all experiments (all methods, 10 times)
Uncomment cell below to launch experiments

In [12]:
# %run -i ..\wann\kin_experiments